In [1]:
from pymilvus import MilvusClient

host = "localhost"
port = 19531
uri = f"http://{host}:{port}"
client = MilvusClient(uri=uri)

In [2]:
chunk_id = 459427222980660815
collection_name = "vdb_test"

response1 = client.get(
            collection_name=collection_name,
            ids=[chunk_id],
        )

In [4]:
%timeit response1 = client.get(collection_name=collection_name, ids=[chunk_id])

400 ms ± 467 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit response2 = client.query(collection_name=collection_name, filter=f"_id == {chunk_id}", limit=1)

400 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
INDEX_PARAMS = [
    {
        "metric_type": "BM25",
        "index_type": "SPARSE_INVERTED_INDEX",
    },  # For sparse vector
    {
        "metric_type": "COSINE",
        "index_type": "HNSW",
        "params": {"M": 32, "efConstruction": 100},
    },  # For dense vector
]


SEARCH_PARAMS = [
        {
            "metric_type": "COSINE",
            "params": {
                "ef": 20,
                "radius": 0.5,
                "range_filter": 1.0,
            },
        },
        {"metric_type": "BM25", "params": {"drop_ratio_build": 0.2}},
    ]


In [24]:
from langchain_milvus import BM25BuiltInFunction, Milvus
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

EMBEDDER_MODEL_NAME = os.environ.get("EMBEDDER_MODEL_NAME", 'Qwen/Qwen3-Embedding-0.6B')
EMBEDDER_BASE_URL = os.environ.get("EMBEDDER_BASE_URL")
EMBEDDER_API_KEY = os.environ.get("EMBEDDER_API_KEY")

uri = f"http://{host}:{port}"

embeddings = OpenAIEmbeddings(
            model=EMBEDDER_MODEL_NAME,
            base_url=EMBEDDER_BASE_URL,
            api_key=EMBEDDER_API_KEY,
        )

sparse_embeddings = BM25BuiltInFunction(enable_match=True)

vector_store = Milvus(
    connection_args={"uri": uri},
    collection_name=collection_name,
    embedding_function=embeddings,
    auto_id=True,
    index_params=INDEX_PARAMS,
    primary_field="_id",
    enable_dynamic_field=True,
    partition_key_field="partition",
    builtin_function=sparse_embeddings,
    vector_field=["sparse", "vector"],
    consistency_level="Strong",
)

In [29]:
%timeit vector_store.similarity_search_with_score(query='Quels sont les activités de Linagora', k=20, fetch_k=20, ranker_type="rrf", ranker_params={"k": 100}, param=SEARCH_PARAMS)

429 ms ± 70.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
